# Plotting

Notebook dedicated to plotting the results of the explainer calculations

In [2]:
# import libraries

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from scipy.integrate import odeint, solve_ivp
from scipy.fft import fft


import matplotlib.pylab as plt
import seaborn as sns
import mpl_interactions.ipyplot as iplt
sns.set_theme(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)

# Machine Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import Sequence
from tensorflow import keras

import shap as shap
try:
    import lime
    import lime.lime_tabular    
except ImportError:
    pass

# Enable Jupyter Notebook's intellisense
%config IPCompleter.greedy=True
%matplotlib inline

%matplotlib notebook
from ipywidgets import *
import matplotlib
import matplotlib.pyplot as plt

# for reproducibility of this notebook:
rng = np.random.RandomState(42)
#tf.random.set_seed(42)
np.random.seed(42)

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#matplotlib.use('TkAgg')


from sklearn.model_selection import ParameterGrid


In [3]:
# choose parameter configuration

alpha = 1.0
beta = -1.0
gamma = 0.37
delta = 0.3
omega = 1.2

parameters = {'alpha': alpha, 'beta': beta, 'gamma': gamma, 'delta': delta, 'omega':omega}

suffix = "params_"+str(parameters['alpha'])+"_"+str(parameters['beta'])+"_"+str(parameters['gamma'])+"_"+str(parameters['delta'])+"_"+str(parameters['omega'])



In [5]:
# load plotting data

data_df = pd.read_csv("Results/explainer_dataframe_"+suffix+".csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Results/explainer_dataframe_params_1.0_-1.0_0.37_0.3_1.2.csv'

In [ ]:
history = pickle.load(open('/trainHistoryDict', "rb"))

In [8]:
with open('/trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

NameError: name 'history' is not defined

In [ ]:
# load neural network

model = tf.keras.models.load_model("Models/ml_model_"+suffix)
history = model.history

In [ ]:
# evaluate NN performance

"""
Evaluate Model
"""

# evaluate the fitting validation and training losses
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

## Make Prdictions on the Test Dataset
y_pred = pd.DataFrame(model.predict(X_test), columns=['xt','vt'])

pred_norm = np.linalg.norm(y_pred[['xt','vt']].values,axis=1)
true_norm = np.linalg.norm(y_test[['xt','vt']].values,axis=1)
hist_data = np.abs(pred_norm-true_norm)/np.abs(true_norm)
hist_data = pd.DataFrame(hist_data, columns=['norm'])

def Remove_Outlier_Indices(df):
    Q1 = df.quantile(0.00)
    Q3 = df.quantile(0.95)
    IQR = Q3 - Q1
    trueList = ~((df > (Q3 + 1.5 * IQR)))
    #trueList = ~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)))
    return trueList

indices = Remove_Outlier_Indices(hist_data)
hist_data = hist_data[indices]

In [ ]:
# Model Accuracy
f, axs = plt.subplots(1, 1, figsize=(4, 4), gridspec_kw=dict(width_ratios=[4]))

sns.lineplot(data = history.history, x = epochs, y='loss',ax=axs, label='loss')
sns.lineplot(data = history.history, x = epochs, y='val_loss',ax=axs, label='val_loss')

axs.set_xlabel('Epochs')
axs.set_ylabel('Loss')
axs.set_title("Losses by Epoch")
axs.legend()


In [ ]:
# Model Error

f, axs = plt.subplots(1, 1, figsize=(4, 4), gridspec_kw=dict(width_ratios=[4]))

# Error Plot for ML Predictions
sns.histplot(data=hist_data, x = 'norm', kde=False, stat='probability', binwidth=0.01, ax=axs)

axs.set(xlim=(0, 0.6), ylim=(0, 0.4))
axs.set_xlabel('Error')
axs.set_ylabel('Probability')
axs.set_title(r"Error Plot $(\frac{||y_{pred}-y_{true}||}{||y_{true}||})$")



In [ ]:
# Loss Plot ML Model
f, axs = plt.subplots(2, 1, figsize=(8, 4), gridspec_kw=dict(width_ratios=[4, 4]))


# True Values Plot
sns.scatterplot(data = y_test.iloc[:,:], x = 'xt', y='vt',ax=axs[0],label='true_values', marker='x', linewidth = 1)

axs[1,0].set(xlim=(-2, 2), ylim=(-2, 2))
axs[1,0].set_xlabel('x [ ]')
axs[1,0].set_ylabel('v [ ]')
axs[1,0].set_title("Phase Space Plot (True Model)")

# ML Values Plot
sns.scatterplot(data = y_pred.iloc[:2000,:], x='xt', y='vt',ax=axs[1],label='pred_values',  marker='x', linewidth = 1)

axs[1,1].set(xlim=(-2, 2), ylim=(-2, 2))
axs[1,1].set_xlabel('x [ ]')
axs[1,1].set_ylabel('v [ ]')
axs[1,1].set_title("Phase Space Plot (ML Model)")

f.tight_layout()

f.savefig("Images/model_summary"+suffix+".svg", dpi='figure')

In [1]:
# individual feature attributions

# Individual Feature Contributions
f, axs = plt.subplots(2, 4, figsize = (16,8), gridspec_kw = dict(width_ratios = [4,4,4,4]))

sns.scatterplot(x='x0', y="xt_x0", data=big_df, ax=axs[0,0], marker = 'x', linewidth = 1, hue = "explainer")

axs[0,0].set_title(r"$x_0$")
axs[0,0].set_xlabel('Index [ ]')
axs[0,0].set_ylabel('Feature Contribution [ ]')

sns.scatterplot(x='v0', y="xt_v0", data=big_df, ax=axs[1,0], marker = 'x', linewidth = 1, hue = "explainer")
axs[1,0].set_title(r"$v_0$")
axs[1,0].set_xlabel('Index [ ]')
axs[1,0].set_ylabel('Feature Contribution [ ]')

sns.scatterplot(x='t', y="xt_t", data=big_df, ax=axs[0,1], marker = 'x', linewidth = 1, hue = "explainer")
axs[0,1].set_title(r"$t$")
axs[0,1].set_xlabel('Index [ ]')
axs[0,1].set_ylabel('Feature Contribution [ ]')

sns.scatterplot(x='rand', y="xt_rand", data=big_df, ax=axs[1,1], marker = 'x', linewidth = 1, hue = "explainer")

axs[1,1].set_title(r"$rand$")
axs[1,1].set_xlabel('Index [ ]')
axs[1,1].set_ylabel('Feature Contribution [ ]')


sns.scatterplot(x='x0', y="vt_x0", data=big_df, ax=axs[0,2], marker = 'x', linewidth = 1, hue = "explainer")

axs[0,2].set_title(r"$x_0$")
axs[0,2].set_xlabel('Index [ ]')
axs[0,2].set_ylabel('Feature Contribution [ ]')

sns.scatterplot(x='v0', y="vt_v0", data=big_df, ax=axs[1,2], marker = 'x', linewidth = 1, hue = "explainer")
axs[1,2].set_title(r"$v_0$")
axs[1,2].set_xlabel('Index [ ]')
axs[1,2].set_ylabel('Feature Contribution [ ]')

sns.scatterplot(x='v0', y="vt_t", data=big_df, ax=axs[0,3], marker = 'x', linewidth = 1, hue = "explainer")
axs[0,3].set_title(r"$t$")
axs[0,3].set_xlabel('Index [ ]')
axs[0,3].set_ylabel('Feature Contribution [ ]')

sns.scatterplot(x='rand', y="vt_rand", data=big_df, ax=axs[1,3], marker = 'x', linewidth = 1, hue = "explainer")
axs[1,3].set_title(r"$rand$")
axs[1,3].set_xlabel('Index [ ]')
axs[1,3].set_ylabel('Feature Contribution [ ]')



f.suptitle(r"Individual Feature Contributions to $x_t$ and $v_t$")

f.tight_layout()
f.savefig("Images/feature_contributions_sorted_by_feature_"+suffix+".svg", dpi='figure')

In [ ]:
# aggregated feature importance

# Aggregated

f, axs = plt.subplots(1, 2, figsize = (16,8), gridspec_kw = dict(width_ratios = [4,4]))


sns.barplot(data=agg_df_2, x = 'feature', y = np.abs(agg_df_2['value']), hue = 'explainer', ax = axs[0], ci = 95, capsize=.2)

axs[0].set_title(r"Aggregate Feature Contributions")
axs[0].set_xlabel('Feature [ ]')
axs[0].set_ylabel('Feature Contribution [ ]')

sns.boxplot(data=agg_df_2, x = 'feature', y = 'value', ax = axs[1], hue = 'explainer')
axs[1].set_title(r"All Feature Contributions")
axs[1].set_xlabel('Feature')
axs[1].set_ylabel('Feature Contribution [ ]')


f.tight_layout()

f.savefig("Images/all_aggregated_"+suffix+".svg", dpi='figure')

In [ ]:
# numerical explainer convergence study

# choose random point from dataset
x_val = np.random.choice(choice, 1)



In [ ]:
class MyLime(shap.other.LimeTabular):
    def __init__(self, model, data, mode="classification"):
        self.model = model
        assert mode in ["classification", "regression"]
        self.mode = mode

        if str(type(data)).endswith("pandas.core.frame.DataFrame'>"):
            data = data.values
        self.data = data
        self.explainer = lime.lime_tabular.LimeTabularExplainer(data, mode=mode)
        self.out_dim = 1#self.model(data[0:1]).shape[1]
            
    def attributions(self, X, num_samples=500, num_features=None):
        try:
            num_features = X.shape[1] if num_features is None else num_features
        except:
            print('exception')
            num_features = 1
        if str(type(X)).endswith("pandas.core.frame.DataFrame'>"):
            X = X.values
            
        out = [np.zeros(X.shape) for j in range(len(self.model))]
        for i in tqdm(range(X.shape[0]), desc="Calculating Lime…", ascii=False, ncols=75):
            exp1 = self.explainer.explain_instance(X[i], self.model[0], labels=range(self.out_dim), 
                                                    num_features=num_features, num_samples=num_samples)
            exp2 = self.explainer.explain_instance(X[i], self.model[1], labels=range(self.out_dim), 
                                                    num_features=num_features, num_samples=num_samples)
            for k, v in exp1.local_exp[1]: 
                out[0][i,k] = v
            for k, v in exp2.local_exp[1]: 
                out[1][i,k] = v
          
        return out

In [ ]:
# lime convergence study
# use x_val from above



In [ ]:
    def __init__(self,
                 training_data,
                 mode="classification",
                 training_labels=None,
                 feature_names=None,
                 categorical_features=None,
                 categorical_names=None,
                 kernel_width=None,
                 kernel=None,
                 verbose=False,
                 class_names=None,
                 feature_selection='auto',
                 discretize_continuous=True,
                 discretizer='quartile',
                 sample_around_instance=False,
                 random_state=None,
                 training_data_stats=None):